In [1]:
import json
import statistics as s
from bert_score import score
from tqdm.notebook import tqdm

%load_ext jupyter_black

In [2]:
# load in combined caption data
captioned_data = None
filepath = "../data/study-2-output/labeled-data/combined-caption-output/combined-caption-output_7304-images_2025-03-29_21:40:00.json"
with open(
    filepath,
    "r",
) as f:
    captioned_data = json.load(f)
captioned_data[0:1]

[{'image_id': 1,
  'file_name': 'VizWiz_train_00000001.jpg',
  'vizwiz_url': 'https://vizwiz.cs.colorado.edu/VizWiz_visualization_img/VizWiz_train_00000001.jpg',
  'text_detected': True,
  'unrecognizable': 0,
  'framing': 0,
  'blur': 5,
  'obstruction': 0,
  'rotation': 0,
  'too dark': 0,
  'too bright': 0,
  'other': 0,
  'no issue': 0,
  'human_captions': [{'caption': 'A can of Coca Cola on a counter is shown for when one can use a nice, cold drink.',
    'is_precanned': False,
    'is_rejected': False},
   {'caption': 'A black can of Coca Cola Zero calorie soda is on the counter near the coffee maker.',
    'is_precanned': False,
    'is_rejected': False},
   {'caption': 'A kitchen counter the various items on top including a can of Coca-Cola, metal containers, and a teapot.',
    'is_precanned': False,
    'is_rejected': False},
   {'caption': 'a black tin of Coca Cola placed on a black surface',
    'is_precanned': False,
    'is_rejected': False},
   {'caption': 'Black counter

In [3]:
def compute_bertscore(dataset, model_type="microsoft/deberta-xlarge-mnli", lang="en"):
    """
    Computes BERTScore for each model against human captions as reference.
    Saves precision, recall, and f1 scores by reference to original input.
    """
    for image in tqdm(dataset):
        curr_references = [
            caption["caption"]
            for caption in image["human_captions"]
            if caption["caption"]
            != "Quality issues are too severe to recognize visual content."
        ]
        curr_output = {}
        for model in image["model_captions"]:
            curr_model_name = model["model_name"]

            # compute scores for current model
            curr_candidate = [model["caption"]]
            P, R, F1 = score(
                curr_candidate, [curr_references], model_type=model_type, lang=lang
            )
            curr_output[curr_model_name] = {
                "scores": {
                    "precision": float(P[0]),
                    "recall": float(R[0]),
                    "f1": float(F1[0]),
                }
            }

        # check if evaluation exists and save score
        if "evaluation" not in image:
            image["evaluation"] = {}
        image["evaluation"]["bertscore"] = curr_output

In [4]:
def compute_average_metrics_bertscore(dataset):
    """
    Computes average precision, recall, and f1 for BERTScore for each model.
    """
    total_scores = {}
    for image in tqdm(dataset):
        curr_evaluation = image["evaluation"]["bertscore"]
        for model_name, scores in curr_evaluation.items():
            scores = scores["scores"]
            if model_name in total_scores:
                total_scores[model_name] = {
                    "total_count": total_scores[model_name]["total_count"] + 1,
                    "total_precision": total_scores[model_name]["total_precision"]
                    + scores["precision"],
                    "total_recall": total_scores[model_name]["total_recall"]
                    + scores["recall"],
                    "total_f1": total_scores[model_name]["total_f1"] + scores["f1"],
                }
            else:
                total_scores[model_name] = {
                    "total_count": 1,
                    "total_precision": scores["precision"],
                    "total_recall": scores["recall"],
                    "total_f1": scores["f1"],
                }

    # compute averages and f1
    output = {}
    for model_name, values in total_scores.items():
        output[model_name] = {
            "avg_precision": values["total_precision"] / float(values["total_count"]),
            "avg_recall": values["total_recall"] / float(values["total_count"]),
            "avg_f1": values["total_f1"] / float(values["total_count"]),
        }
        output[model_name]["f1"] = s.harmonic_mean(
            [output[model_name]["avg_precision"], output[model_name]["avg_recall"]]
        )

    return output

In [5]:
compute_bertscore(captioned_data[0:10])

  0%|          | 0/10 [00:00<?, ?it/s]

In [6]:
print(json.dumps(compute_average_metrics_bertscore(captioned_data[0:10]), indent=4))

  0%|          | 0/10 [00:00<?, ?it/s]

{
    "gpt-4o-2024-08-06": {
        "avg_precision": 0.6272800266742706,
        "avg_recall": 0.780607956647873,
        "avg_f1": 0.6917834937572479,
        "f1": 0.6955947996839815
    },
    "Llama-3.2-11B-Vision-Instruct": {
        "avg_precision": 0.6076797366142273,
        "avg_recall": 0.760029011964798,
        "avg_f1": 0.6693472445011139,
        "f1": 0.6753692703797957
    },
    "Molmo-7B-O-0924": {
        "avg_precision": 0.6660487532615662,
        "avg_recall": 0.7902890324592591,
        "avg_f1": 0.7159294486045837,
        "f1": 0.7228694193706547
    }
}
